# 机器翻译
- 编码器--解码器
- 注意力机制
- 主要步骤
    - 1 读取和预处理数据
    - 2 注意力机制的编码器解码器(模型)
    - 3 训练模型
    - 4 预测不定长序列
    - 5 评价翻译结果
    
## 1 读取和预处理数据
- 我们先定义一些特殊符号。其中$<pad>$（padding）符号用来添加在较短序列后，直到每个序列等长，而$<bos>$和$<eos>$符号分别表示序列的开始和结束。

In [2]:
import collections
import os
import io
import math
import torch
from torch import nn
import torch.nn.functional as F
import torchtext.vocab as Vocab
import torch.utils.data as Data

import sys
sys.path.append("..") 
# import d2lzh_pytorch as d2l

PAD, BOS, EOS = '<pad>', '<bos>', '<eos>'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 辅助函数预处理数据

In [3]:
# 将一个序列中所有的词记录在all_tokens中以便之后构造词典，然后在该序列后面添加PAD直到序列
# 长度变为max_seq_len，然后将序列保存在all_seqs中
def process_one_seq(seq_tokens, all_tokens, all_seqs, max_seq_len):
    all_tokens.extend(seq_tokens)
    seq_tokens += [EOS] + [PAD] * (max_seq_len - len(seq_tokens) - 1)
    all_seqs.append(seq_tokens)

# 使用所有的词来构造词典。并将所有序列中的词变换为词索引后构造Tensor
def build_data(all_tokens, all_seqs):
    vocab = Vocab.Vocab(collections.Counter(all_tokens),
                        specials=[PAD, BOS, EOS])
    indices = [[vocab.stoi[w] for w in seq] for seq in all_seqs]
    return vocab, torch.tensor(indices)

### 读取数据/构建词典
- 为了演示方便，我们在这里使用一个很小的法语—英语数据集。在这个数据集里，每一行是一对法语句子和它对应的英语句子，中间使用'\t'隔开。在读取数据时，我们在句末附上$<eos>$符号，并可能通过添加$<pad>$符号使每个序列的长度均为**max_seq_len**。我们为法语词和英语词分别创建词典。法语词的索引和英语词的索引相互独立。

In [4]:
def read_data(max_seq_len):
    # in和out分别是input和output的缩写
    in_tokens, out_tokens, in_seqs, out_seqs = [], [], [], []
    with io.open('data/fr-en-small.txt') as f:
        lines = f.readlines()
    for line in lines:
        in_seq, out_seq = line.rstrip().split('\t')
        in_seq_tokens, out_seq_tokens = in_seq.split(' '), out_seq.split(' ')
        if max(len(in_seq_tokens), len(out_seq_tokens)) > max_seq_len - 1:
            continue  # 如果加上EOS后长于max_seq_len，则忽略掉此样本
        process_one_seq(in_seq_tokens, in_tokens, in_seqs, max_seq_len)
        process_one_seq(out_seq_tokens, out_tokens, out_seqs, max_seq_len)
    in_vocab, in_data = build_data(in_tokens, in_seqs)
    out_vocab, out_data = build_data(out_tokens, out_seqs)
    return in_vocab, out_vocab, Data.TensorDataset(in_data, out_data)

In [5]:
# 将序列的最大长度设成7，然后查看读取到的第一个样本。该样本分别包含法语词索引序列和英语词索引序列。
max_seq_len = 7
in_vocab, out_vocab, dataset = read_data(max_seq_len)
dataset[0]

(tensor([ 5,  4, 45,  3,  2,  0,  0]), tensor([ 8,  4, 27,  3,  2,  0,  0]))

## 2 模型
- 使用含注意力机制的编码器—解码器来将一段简短的法语翻译成英语。下面我们来介绍模型的实现。
### 编码器
- 在编码器中，我们将输入语言的词索引通过词嵌入层得到词的表征，然后输入到一个多层门控循环单元中。正如我们在6.5节（循环神经网络的简洁实现）中提到的，PyTorch的nn.GRU实例在前向计算后也会分别返回输出和最终时间步的多层隐藏状态。其中的输出指的是最后一层的隐藏层在各个时间步的隐藏状态，并不涉及输出层计算。注意力机制将这些输出作为键项和值项。

In [6]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 drop_prob=0, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers, dropout=drop_prob)

    def forward(self, inputs, state):
        # 输入形状是(批量大小, 时间步数)。将输出互换样本维和时间步维
        embedding = self.embedding(inputs.long()).permute(1, 0, 2) # (seq_len, batch, input_size)
        return self.rnn(embedding, state)

    def begin_state(self):
        return None # 隐藏态初始化为None时PyTorch会自动初始化为0

### 
---
- 下面我们来创建一个批量大小为4、时间步数为7的小批量序列输入。设门控循环单元的隐藏层个数为2，隐藏单元个数为16。编码器对该输入执行前向计算后返回的输出形状为(时间步数, 批量大小, 隐藏单元个数)。门控循环单元在最终时间步的多层隐藏状态的形状为(隐藏层个数, 批量大小, 隐藏单元个数)。对于门控循环单元来说，state就是一个元素，即隐藏状态；如果使用长短期记忆，state是一个元组，包含两个元素即隐藏状态和记忆细胞。


In [7]:
encoder = Encoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
output, state = encoder(torch.zeros((4, 7)), encoder.begin_state())
output.shape, state.shape # GRU的state是h, 而LSTM的是一个元组(h, c)

(torch.Size([7, 4, 16]), torch.Size([2, 4, 16]))

### 注意力机制
- 我们将实现注意力机制.定义的函数aa：将输入连结后通过含单隐藏层的多层感知机变换。其中隐藏层的输入是解码器的隐藏状态与编码器在所有时间步上隐藏状态的一一连结，且使用tanh函数作为激活函数。输出层的输出个数为1。两个Linear实例均不使用偏差。其中函数aa定义里向量vv的长度是一个超参数，即attention_size。

In [8]:
def attention_model(input_size, attention_size):
    model = nn.Sequential(nn.Linear(input_size, 
                                    attention_size, bias=False),
                          nn.Tanh(),
                          nn.Linear(attention_size, 1, bias=False))
    return model

- 注意力机制的输入包括查询项、键项和值项。设编码器和解码器的隐藏单元个数相同。这里的查询项为解码器在上一时间步的隐藏状态，形状为(批量大小, 隐藏单元个数)；键项和值项均为编码器在所有时间步的隐藏状态，形状为(时间步数, 批量大小, 隐藏单元个数)。注意力机制返回当前时间步的背景变量，形状为(批量大小, 隐藏单元个数)。

In [9]:
def attention_forward(model, enc_states, dec_state):
    """
    enc_states: (时间步数, 批量大小, 隐藏单元个数)
    dec_state: (批量大小, 隐藏单元个数)
    """
    # 将解码器隐藏状态广播到和编码器隐藏状态形状相同后进行连结
    dec_states = dec_state.unsqueeze(dim=0).expand_as(enc_states)
    enc_and_dec_states = torch.cat((enc_states, dec_states), dim=2)
    e = model(enc_and_dec_states)  # 形状为(时间步数, 批量大小, 1)
    alpha = F.softmax(e, dim=0)  # 在时间步维度做softmax运算
    return (alpha * enc_states).sum(dim=0)  # 返回背景变量

- 在下面的例子中，编码器的时间步数为10，批量大小为4，编码器和解码器的隐藏单元个数均为8。注意力机制返回一个小批量的背景向量，每个背景向量的长度等于编码器的隐藏单元个数。因此输出的形状为(4, 8)。

In [10]:
seq_len, batch_size, num_hiddens = 10, 4, 8
model = attention_model(2*num_hiddens, 10) 
enc_states = torch.zeros((seq_len, batch_size, num_hiddens))
dec_state = torch.zeros((batch_size, num_hiddens))
attention_forward(model, enc_states, dec_state).shape # torch.Size([4, 8])

torch.Size([4, 8])

### 含注意力机制的解码器
- 我们直接将编码器在最终时间步的隐藏状态作为解码器的初始隐藏状态。这要求编码器和解码器的循环神经网络使用相同的隐藏层个数和隐藏单元个数。

- 在解码器的前向计算中，我们先通过刚刚介绍的注意力机制计算得到当前时间步的背景向量。由于解码器的输入来自输出语言的词索引，我们将输入通过词嵌入层得到表征，然后和背景向量在特征维连结。我们将连结后的结果与上一时间步的隐藏状态通过门控循环单元计算出当前时间步的输出与隐藏状态。最后，我们将输出通过全连接层变换为有关各个输出词的预测，形状为(批量大小, 输出词典大小)。

In [11]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 attention_size, drop_prob=0):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.attention = attention_model(2*num_hiddens, attention_size)
        # GRU的输入包含attention输出的c和实际输入, 所以尺寸是 num_hiddens+embed_size
        self.rnn = nn.GRU(num_hiddens + embed_size, num_hiddens, 
                          num_layers, dropout=drop_prob)
        self.out = nn.Linear(num_hiddens, vocab_size)

    def forward(self, cur_input, state, enc_states):
        """
        cur_input shape: (batch, )
        state shape: (num_layers, batch, num_hiddens)
        """
        # 使用注意力机制计算背景向量
        c = attention_forward(self.attention, enc_states, state[-1])
        # 将嵌入后的输入和背景向量在特征维连结, (批量大小, num_hiddens+embed_size)
        input_and_c = torch.cat((self.embedding(cur_input), c), dim=1)
        # 为输入和背景向量的连结增加时间步维，时间步个数为1
        output, state = self.rnn(input_and_c.unsqueeze(0), state)
        # 移除时间步维，输出形状为(批量大小, 输出词典大小)
        output = self.out(output).squeeze(dim=0)
        return output, state

    def begin_state(self, enc_state):
        # 直接将编码器最终时间步的隐藏状态作为解码器的初始隐藏状态
        return enc_state

## 3 训练模型
- 我们先实现batch_loss函数计算一个小批量的损失。解码器在最初时间步的输入是特殊字符BOS。之后，解码器在某时间步的输入为样本输出序列在上一时间步的词，即强制教学。此外，同10.3节（word2vec的实现）中的实现一样，我们在这里也使用掩码变量避免填充项对损失函数计算的影响。

In [21]:
def batch_loss(encoder, decoder, X, Y, loss):
    batch_size = X.shape[0]
    enc_state = encoder.begin_state()
    enc_outputs, enc_state = encoder(X, enc_state)
    # 初始化解码器的隐藏状态
    dec_state = decoder.begin_state(enc_state)
    # 解码器在最初时间步的输入是BOS
    dec_input = torch.tensor([out_vocab.stoi[BOS]] * batch_size)
    # 我们将使用掩码变量mask来忽略掉标签为填充项PAD的损失
    mask, num_not_pad_tokens = torch.ones(batch_size,), 0
    l = torch.tensor([0.0])
    for y in Y.permute(1,0): # Y shape: (batch, seq_len)
        dec_output, dec_state = decoder(dec_input, dec_state, enc_outputs)
        print("dec_output:", dec_output, dec_output.size())
#         print("dec_state:", dec_state, dec_state.size())
        print("y", y)
        print("CrossEntropyLoss:", loss(dec_output, y))
        print("mask", mask)
        print("-"*30)
        l = l + (mask * loss(dec_output, y)).sum()
        dec_input = y  # 使用强制教学
        num_not_pad_tokens += mask.sum().item()
        # EOS后面全是PAD. 下面一行保证一旦遇到EOS接下来的循环中mask就一直是0
        mask = mask * (y != out_vocab.stoi[EOS]).float()
    return l / num_not_pad_tokens

- 在训练函数中，我们需要同时迭代编码器和解码器的模型参数。

In [22]:
def train(encoder, decoder, dataset, lr, batch_size, num_epochs):
    enc_optimizer = torch.optim.Adam(encoder.parameters(), lr=lr)
    dec_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)

    loss = nn.CrossEntropyLoss(reduction='none')
    data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)
    for epoch in range(num_epochs):
        l_sum = 0.0
        for X, Y in data_iter:
            enc_optimizer.zero_grad()
            dec_optimizer.zero_grad()
            l = batch_loss(encoder, decoder, X, Y, loss)
            l.backward()
            enc_optimizer.step()
            dec_optimizer.step()
            l_sum += l.item()
            break
        if (epoch + 1) % 10 == 0:
            print("epoch %d, loss %.3f" % (epoch + 1, l_sum / len(data_iter)))

- 创建模型实例并设置超参数。然后，我们就可以训练模型了。

In [23]:
embed_size, num_hiddens, num_layers = 64, 64, 2
attention_size, drop_prob, lr, batch_size, num_epochs = 10, 0.5, 0.01, 2, 50
encoder = Encoder(len(in_vocab), embed_size, num_hiddens, num_layers,
                  drop_prob)
decoder = Decoder(len(out_vocab), embed_size, num_hiddens, num_layers,
                  attention_size, drop_prob)
train(encoder, decoder, dataset, lr, batch_size, num_epochs)

dec_output: tensor([[-0.0530,  0.0343, -0.2249,  0.0846,  0.0533, -0.0099,  0.1071,  0.0626,
          0.0067, -0.0380,  0.0948, -0.0791,  0.2040,  0.0147,  0.1740, -0.0372,
          0.0312,  0.2790, -0.1234,  0.0989, -0.1049,  0.1185, -0.1320,  0.0996,
         -0.1104, -0.0279, -0.0308,  0.0174,  0.0824,  0.0717,  0.0279,  0.0635,
          0.0303,  0.0029, -0.1064,  0.2309,  0.1682, -0.1782],
        [-0.0775, -0.0429, -0.0777,  0.0881,  0.0947,  0.1982,  0.0187,  0.1557,
         -0.0254, -0.0975,  0.0561, -0.0324,  0.1255, -0.0226,  0.0260, -0.0714,
         -0.0030,  0.2180, -0.0288,  0.0683, -0.0644,  0.0387,  0.0220,  0.1023,
         -0.1515,  0.0675, -0.0315,  0.0614,  0.1193,  0.0855,  0.0044,  0.0585,
          0.0428, -0.0835, -0.1310,  0.1250,  0.1615, -0.0856]],
       grad_fn=<SqueezeBackward1>) torch.Size([2, 38])
y tensor([8, 8])
CrossEntropyLoss: tensor([3.6607, 3.6911], grad_fn=<NllLossBackward>)
mask tensor([1., 1.])
------------------------------
dec_output: tens

dec_output: tensor([[-1.3427, -2.0214, -0.6106, -0.2388,  3.4663, -0.4521,  3.2005, -0.1715,
          3.6586,  0.0954,  0.2972, -0.0562, -0.4985, -0.4664, -1.4699, -0.8856,
         -1.2935, -0.3908, -1.3018, -1.1857, -1.0686, -1.6594, -1.3451, -0.4488,
         -0.2221,  0.3257, -0.8510, -0.2738,  0.0666, -0.5353, -0.1497, -1.3908,
         -0.6090,  0.3799, -0.9214, -0.9139, -1.1959, -0.7582],
        [-1.2786, -1.7694, -0.8453, -0.4557,  3.5723, -0.1190,  3.1219,  0.1409,
          3.7781, -0.1628,  0.5030, -0.1720, -0.5479, -0.4377, -1.3665, -0.9574,
         -1.2889, -0.2639, -1.1957, -1.1483, -0.9788, -1.6376, -0.9535, -0.2582,
         -0.2755,  0.4096, -0.9445, -0.2834, -0.0310, -0.5531,  0.1099, -1.3749,
         -0.7966,  0.3799, -0.9651, -0.7324, -1.0644, -0.7461]],
       grad_fn=<SqueezeBackward1>) torch.Size([2, 38])
y tensor([8, 7])
CrossEntropyLoss: tensor([1.0994, 4.6801], grad_fn=<NllLossBackward>)
mask tensor([1., 1.])
------------------------------
dec_output: tens

dec_output: tensor([[-0.5507, -0.9845, -1.0684,  2.3081, -0.4438,  1.2443, -0.8781, -0.2565,
         -1.6878, -0.0645, -0.3471, -0.4453,  0.5410, -0.6287,  1.8275, -0.0487,
         -0.1981, -0.8378, -0.6355, -0.3356, -0.3575, -0.4472, -0.4486, -0.0204,
         -0.0318, -0.1978, -0.1863,  0.2600, -1.1351, -0.0450, -0.4953,  1.2900,
         -0.4078, -0.4344, -0.3701, -0.2605,  0.4614,  0.2792],
        [-0.4541, -0.9719, -1.3014,  2.3095, -0.4820,  1.4100, -0.7273, -0.0583,
         -1.5571, -0.3172, -0.3106, -0.2961,  0.3962, -0.6096,  2.1456, -0.2403,
         -0.1439, -0.6192, -0.6156, -0.2736, -0.4358, -0.5349, -0.4192, -0.0100,
         -0.2252, -0.0605, -0.1088,  0.3817, -1.1976,  0.1670, -0.4216,  1.6842,
         -0.4006, -0.4575, -0.4186, -0.1823,  0.5927,  0.2220]],
       grad_fn=<SqueezeBackward1>) torch.Size([2, 38])
y tensor([23, 14])
CrossEntropyLoss: tensor([3.9146, 1.8497], grad_fn=<NllLossBackward>)
mask tensor([1., 1.])
------------------------------
dec_output: te

dec_output: tensor([[-2.6737, -3.1787,  2.9168,  6.4708, -1.7979, -1.8114, -2.8264, -3.0070,
         -3.7249, -0.7527, -0.1858, -0.9311,  1.0658, -1.3004,  1.4482, -1.9392,
         -2.5604, -1.1661, -3.2193, -0.7847,  0.2546, -2.5060, -0.4499, -0.7948,
         -1.2586, -0.4026, -0.6637, -0.6328, -0.9044, -0.5802, -2.0622,  0.0854,
         -0.4588, -0.2699,  0.0303, -2.5202, -1.2477,  0.5669],
        [-2.5912, -3.0774,  3.4200,  6.2644, -1.8289, -2.2459, -2.7988, -2.9401,
         -3.7561, -0.2211, -0.2763, -0.8062,  1.0412, -1.4571,  0.9891, -2.1237,
         -2.4218, -1.0898, -3.1823, -0.6755,  0.2264, -2.7117, -1.0108, -1.0586,
         -0.9054, -0.3102, -0.7713, -0.4730, -0.5029, -0.7630, -2.0762, -0.4116,
         -0.5019, -0.2380,  0.4828, -2.4967, -1.2721,  0.8812]],
       grad_fn=<SqueezeBackward1>) torch.Size([2, 38])
y tensor([3, 3])
CrossEntropyLoss: tensor([0.0607, 0.0941], grad_fn=<NllLossBackward>)
mask tensor([1., 1.])
------------------------------
dec_output: tens

mask tensor([1., 1.])
------------------------------
dec_output: tensor([[-2.3256, -2.3958,  1.5015,  5.6385, -2.6147, -2.4963, -3.5799, -3.5351,
         -3.2504,  0.4122,  0.1990,  0.0152,  0.9135, -1.1935,  1.3152, -1.4833,
         -2.0149, -0.0626, -1.3093, -0.0893,  0.3639, -1.7609, -0.5120, -0.7201,
         -1.0551,  0.1486,  0.3975,  0.0395,  0.9905, -0.2074, -1.4085,  0.7136,
         -0.6166,  0.2701,  0.0943, -0.0812,  0.2810,  1.3513],
        [-2.9168, -3.1518,  7.6415,  4.5709, -1.9057, -1.9087, -3.5218, -3.2418,
         -2.5044, -0.4050, -0.9638,  0.6313,  0.5466, -2.2532,  0.0691, -1.8522,
         -2.9331, -1.9129, -3.5733, -0.9977,  0.1360, -3.6938, -0.2128, -1.5142,
         -1.4384, -0.5815, -0.0219, -0.6785,  0.1592, -1.9052, -2.7656, -1.2531,
          0.1677, -0.3042,  0.1721, -2.3113, -1.3857,  1.4034]],
       grad_fn=<SqueezeBackward1>) torch.Size([2, 38])
y tensor([3, 2])
CrossEntropyLoss: tensor([0.1308, 0.0549], grad_fn=<NllLossBackward>)
mask tensor([1.,

y tensor([2, 2])
CrossEntropyLoss: tensor([0.0253, 0.0064], grad_fn=<NllLossBackward>)
mask tensor([1., 1.])
------------------------------
dec_output: tensor([[-1.3426, -2.0538,  3.3840,  4.6256, -3.3924, -3.1898, -3.1284, -2.4507,
         -1.7785, -0.4334, -0.5592,  0.8565,  0.3400, -1.3496, -0.0409,  0.1332,
         -1.3277, -0.2597, -1.3944, -0.0648,  0.4253, -1.8380, -0.4669, -0.9781,
         -1.6195,  0.2063,  0.3839, -0.1823,  1.5052, -0.8521, -1.9858, -0.6266,
          0.4550,  0.5054,  0.3253,  0.4216,  0.3140,  0.5635],
        [-1.4209, -1.6300,  5.3873,  4.3610, -2.8458, -2.6532, -3.7178, -2.5972,
         -2.3690, -0.4142, -0.7636,  0.7797,  0.5471, -1.5215, -0.2995,  0.2390,
         -1.5846, -1.0747, -1.6664, -0.3021,  0.3966, -2.1320, -0.2013, -0.9456,
         -1.6624,  0.1702,  0.2351, -0.5430,  1.4684, -1.3748, -1.9971, -1.1030,
          0.5252,  0.4436,  0.6396,  0.3316, -0.1736,  0.4293]],
       grad_fn=<SqueezeBackward1>) torch.Size([2, 38])
y tensor([0, 0])

CrossEntropyLoss: tensor([0.0444, 0.1700], grad_fn=<NllLossBackward>)
mask tensor([1., 1.])
------------------------------
dec_output: tensor([[-1.3106, -1.6817,  4.0111,  4.5854, -2.0283, -3.5215, -2.8613, -2.6952,
         -2.3217, -1.0289, -0.3368,  2.3337,  0.4443, -1.6940, -1.2950,  0.5480,
         -1.7422, -0.0863, -1.6841, -0.4970,  0.4167, -2.3821, -0.0915, -1.1064,
         -1.9207, -0.0148,  0.5744, -0.4267,  1.8821, -1.1546, -2.5476, -1.7196,
          2.1652,  0.6307,  0.1086, -0.3561, -0.4889,  0.2780],
        [-1.7695, -2.0216,  7.2519,  1.6813, -0.4779, -3.2402, -1.8239, -1.7973,
         -0.2247,  0.2479, -1.0537,  1.8439,  0.9929, -1.0118, -2.0499,  0.7086,
         -1.7367, -0.6316, -2.8337, -0.3240,  0.0203, -4.3423, -0.7707, -1.9427,
         -1.7387, -0.1536,  1.1678, -0.4232,  1.0891, -1.4550, -2.2559, -3.0707,
          1.7594,  0.1971,  0.9966, -0.8063, -1.8656,  0.2525]],
       grad_fn=<SqueezeBackward1>) torch.Size([2, 38])
y tensor([0, 2])
CrossEntropyLoss

## 4 预测不定长的序列
- 主流有3种方法来生成解码器在每个时间步的输出。这里我们实现最简单的贪婪搜索(greedy search)。

In [17]:
def translate(encoder, decoder, input_seq, max_seq_len):
    in_tokens = input_seq.split(' ')
    in_tokens += [EOS] + [PAD] * (max_seq_len - len(in_tokens) - 1)
    enc_input = torch.tensor([[in_vocab.stoi[tk] for tk in in_tokens]]) # batch=1
    enc_state = encoder.begin_state()
    enc_output, enc_state = encoder(enc_input, enc_state)
    dec_input = torch.tensor([out_vocab.stoi[BOS]])
    dec_state = decoder.begin_state(enc_state)
    output_tokens = []
    for _ in range(max_seq_len):
        dec_output, dec_state = decoder(dec_input, dec_state, enc_output)
        pred = dec_output.argmax(dim=1)
        pred_token = out_vocab.itos[int(pred.item())]
        if pred_token == EOS:  # 当任一时间步搜索出EOS时，输出序列即完成
            break
        else:
            output_tokens.append(pred_token)
            dec_input = pred
    return output_tokens

- 简单测试一下模型。输入法语句子“ils regardent.”，翻译后的英语句子应该是“they are watching.”。

In [19]:
input_seq = 'ils regardent .'
translate(encoder, decoder, input_seq, max_seq_len)

['they', 'are', 'watching', '.']

## 5 评价翻译结果
- bleu

In [20]:
def bleu(pred_tokens, label_tokens, k):
    len_pred, len_label = len(pred_tokens), len(label_tokens)
    score = math.exp(min(0, 1 - len_label / len_pred))
    for n in range(1, k + 1):
        num_matches, label_subs = 0, collections.defaultdict(int)
        for i in range(len_label - n + 1):
            label_subs[''.join(label_tokens[i: i + n])] += 1
        for i in range(len_pred - n + 1):
            if label_subs[''.join(pred_tokens[i: i + n])] > 0:
                num_matches += 1
                label_subs[''.join(pred_tokens[i: i + n])] -= 1
        score *= math.pow(num_matches / (len_pred - n + 1), math.pow(0.5, n))
    return score